In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import folium
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import pickle
import io
import fiona

from PIL import Image
from tqdm.notebook import tqdm
from pyproj import Transformer
from shapely.geometry import LineString, Point

In [11]:
FASKES = 'data/puskesmas_dokter.xlsx'
RESULT = 'data/puskesmas_dokter_ntb.geojson'
BEBAN = 'data/beban.csv'

In [17]:
faskes = pd.read_excel(FASKES, dtype=str)

In [18]:
faskes_ntb = faskes.loc[faskes['prov']=='NUSA TENGGARA BARAT']

In [19]:
amenities_node = gpd.GeoDataFrame(
    faskes_ntb,
    geometry=gpd.points_from_xy(
        faskes_ntb['lng'],
        faskes_ntb['lat']
    )
)
amenities_node = amenities_node.set_crs("EPSG:4326")

In [20]:
beban = pd.read_csv(BEBAN, dtype=str)

In [22]:
amenities_node

,prov,nmkab,kdppk,jnsppk,nmppk,telpppk,nmjlnppk,lat,lng,tglhbspksppk,total_staff,total_doctors,jadwal,jumlahstaf,dokter,geometry
6096,NUSA TENGGARA BARAT,KAB. LOMBOK BARAT,02340001,NaN,SIGERONGAN,0370-7561647,JL. KARANG BAYAN SIGERONGAN,-8.56791,116.153,31-12-2024,36,9,"[{'no': '1', 'nama_hari': 'SENIN', 'buka': '07...","([{'no': '1', 'kdppk': '', 'nama': 'Dokter', '...","([{'no': '1', 'namadokter': 'dr. Ni Made Novi ...",POINT (116.15300 -8.56791)
6097,NUSA TENGGARA BARAT,KAB. LOMBOK BARAT,02341003,NaN,DASAN TAPEN,0370-681553,JL. TGH ABDUSSAMAD GERUNG,-8.66489,116.133,31-12-2024,20,9,"[{'no': '1', 'nama_hari': 'SENIN', 'buka': '07...","([{'no': '1', 'kdppk': '', 'nama': 'Dokter', '...","([{'no': '1', 'namadokter': 'HANA PERMATA SARI...",POINT (116.13300 -8.66489)
6098,NUSA TENGGARA BARAT,KAB. LOMBOK BARAT,02341004,NaN,EYAT MAYANG,0877-30090034,"JL.RAYA BARU SEKOTONG, DESA EYAT MAYANG",-8.766130129339754,116.0743549356493,31-12-2024,31,4,"[{'no': '1', 'nama_hari': 'SENIN', 'buka': '08...","([{'no': '1', 'kdppk': '', 'nama': 'Dokter', '...","([{'no': '1', 'namadokter': 'dr. Nurmah', 'nam...",POINT (116.07435 -8.76613)
6099,NUSA TENGGARA BARAT,KAB. LOMBOK BARAT,02341005,NaN,SURANADI,0370-7563363,"JALAN RAYA SURANADI-SESAOT,DES",-8.57019,116.227,31-12-2024,25,5,"[{'no': '1', 'nama_hari': 'SENIN', 'buka': '07...","([{'no': '1', 'kdppk': '', 'nama': 'Dokter', '...","([{'no': '1', 'namadokter': 'SASKY MS RAHMAN',...",POINT (116.22700 -8.57019)
6100,NUSA TENGGARA BARAT,KAB. LOMBOK BARAT,02341006,NaN,SESELA,0853-37384646,JALAN RAYA SESELA NO.17 KM 1,-8.553571828506303,116.0975708465576,31-12-2024,29,6,"[{'no': '1', 'nama_hari': 'SENIN', 'buka': '08...","([{'no': '1', 'kdppk': '', 'nama': 'Dokter', '...","([{'no': '1', 'namadokter': 'dr. Nugraha Arief...",POINT (116.09757 -8.55357)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6266,NUSA TENGGARA BARAT,KAB. LOMBOK UTARA,23010101,NaN,BAYAN,0-8175707774,JL. RAYA TANJUNG BAYAN,-8.23024,116.417,31-12-2024,11,4,"[{'no': '1', 'nama_hari': 'SENIN', 'buka': '00...","([{'no': '1', 'kdppk': '', 'nama': 'Dokter', '...","([{'no': '1', 'namadokter': 'SRI SANUR ENDAH',...",POINT (116.41700 -8.23024)
6267,NUSA TENGGARA BARAT,KAB. LOMBOK UTARA,23010102,NaN,KAYANGAN,0-08234047,Jl. Raya Kayangan Santong,-8.25935,116.261,31-12-2024,10,3,"[{'no': '1', 'nama_hari': 'SENIN', 'buka': '07...","([{'no': '1', 'kdppk': '', 'nama': 'Dokter', '...","([{'no': '1', 'namadokter': 'dr. Lalu Dedy Rus...",POINT (116.26100 -8.25935)
6268,NUSA TENGGARA BARAT,KAB. LOMBOK UTARA,23010201,NaN,GANGGA,0370-6198926,"JL. RAYA TANJUNG, GONDANG",-8.33549,116.193,31-12-2024,49,5,"[{'no': '1', 'nama_hari': 'SENIN', 'buka': '08...","([{'no': '1', 'kdppk': '', 'nama': 'Dokter', '...","([{'no': '1', 'namadokter': 'dr. I Gede Agus P...",POINT (116.19300 -8.33549)
6269,NUSA TENGGARA BARAT,KAB. LOMBOK UTARA,23010301,NaN,TANJUNG,0370-037062301,"JALAN KEMBANG DANGAR, DUSUN LE",-8.37204,116.156,31-12-2024,14,6,"[{'no': '1', 'nama_hari': 'SENIN', 'buka': '08...","([{'no': '1', 'kdppk': '', 'nama': 'Dokter', '...","([{'no': '1', 'namadokter': 'drg. Qonita', 'na...",POINT (116.15600 -8.37204)


In [14]:
df = pd.merge(amenities_node, beban)

In [15]:
df.to_file(RESULT)

C:\Users\Acer\.conda\envs\geoanalisis\Lib\site-packages\geopandas\io\file.py:610: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


In [16]:
df

,prov,nmkab,kdppk,jnsppk,nmppk,telpppk,nmjlnppk,lat,lng,tglhbspksppk,total_staff,total_doctors,jadwal,jumlahstaf,dokter,geometry,Unnamed: 0,beban_faskes
